<a href="https://colab.research.google.com/github/EstebanmAcero/Neural_nets/blob/main/Regularizacion_y_optimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Luis Esteban Molina Acero 201910203**

### **Manuel Velez**

# Regularización

La regularización es una tecnica para evitar el overfitting en Machine Learning, esto se refiere qa que el modelo aprenda demasiado con los datos de entrenamiento que generalice respuestas para datos de entrada distintos.

Este puede ser validado contrastando las curvas de entrenamiento y validación con respecto a la pérdida.

Por ejemplo si la el error de validación incrementa mientras que el error de entrenamiento se disminuye se puede tener una situación de overfitting.

Se prepara el dataset usando el dataset del iris de la flor de Anderson: el cual contiene 150 registros bajo 5 atributos tales como :    

1. Longitud del sepalo.
2. Ancho del sepalo.
3. Longitud del petalo.
4. Ancho del petalo.
5. Clase o objetivo .


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

Vamos a explorar los datos de manera que se carguen los datos en un DataFrame

In [ ]:
# load data into a dataframe
df = pd.DataFrame(iris.data, columns = iris.feature_names)

# convert datatype to float
df['label'] = iris.target

# use string label instead
df['label'] = df.label.replace(dict(enumerate (iris.target_names)))

In [ ]:
df

Se realiza la transformación con One Hot encoding para pasar de variables categoricas a numericas.

In [ ]:
# label -> one-hot encoding
label = pd.get_dummies(df['label'], prefix = 'label')
df = pd.concat([df, label], axis = 1)

# drop old label
df.drop(['label'], axis = 1, inplace = True)


In [ ]:
df

In [ ]:
#Vamos a crear variables x and y
# Keras and TensorFlow solo aceptan arreglos como entradas
# asi que vamos a convertir el dataframe en un arreglo

X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]

# Convert dataframe into np array
X = np.asarray(X)

y = df[['label_setosa', 'label_versicolor','label_virginica']]

# Convert DataFrame into np array
y = np.asarray(y)

Vamos a separar los datos para el entrenamiento y la prueba usando

"train_test_spit() "


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


# Vamos a construir el model de red neuronal sin regularizacion

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ])
    return model

# la primer capa tiene un tamaño de 4, de 64 unidades seguida por tres capas
# de 128 unidades además hay 3 capas de 64 unidades densas
# todas con la función de activación RELU
# la capa de salida tiene 3 unidades y una función softmax de activacíón


In [ ]:
# Vamos a correr el modelo

model = create_model()
model.summary()  # se crea un resumen del modelo

Se procede a configurar el modelo con compile() pasando los siguientes argumentos:


* adam : un algoritmo optimizador.
* categorical_corssentropy : para un problema de clasificación múltiple.
* acurracy : para tener métricas de evaluación para el entranamiento y prueba.


In [ ]:
model.compile(  optimizer  ='adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
#model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'acuracy')
#model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'acurracy',  run_eagerly=True)


In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)


In [ ]:
# utilizamos model.fit() acomodar el modelo a los datos de entrenamiento
history = model.fit(x_train, y_train, epochs=200, validation_split=0.25, batch_size=40, verbose=2)
#history = model.fit( x_train, y_train, epochs = 200, validation_split = 0.25, batch_size = 40, verbose = 2)

# Se evalua el modelo una vez que se ha entrenado el modelo por lo que implica:

1. Una gráfica de las métricas de pérdida y de acurracy.
2. Probar el modelo contra datso de entrenamiento nunca usados.

Por lo anterior se crea la funcion plot_metric().



In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def plot_metric(history, metric) :
    train_metrics  = history.history [metric]
    val_metrics    = history.history ['val_'+ metric]
    epochs         = range (1, len(train_metrics) + 1)
    plt.plot (epochs, train_metrics)
    plt.plot (epochs, val_metrics)
    plt.title('Training and valitation ' + metric)
    plt.xlabel ("Epochs")
    plt.ylabel (metric)
    plt.legend(["train_"+metric, 'val_'+ metric])

    plt.show()

plot_metric(history, 'categorical_accuracy')




In [ ]:
plot_metric(history, 'loss')

In [ ]:
# Vamos a evaluar el modelo con el set de prueba

model.evaluate (x_test, y_test, verbose = 2)

# Añadiendo la regularización L2 y Dropout



In [ ]:
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.regularizers  import l2


Vamos a crear una funccion llamada _create_regularized_model_() la cual retorna un modelo similar al construido pero esta vez añadira Regularización L2 y capas Dropout, por lo que está función toma dos argumentos:

1. El factor de regularización L2.
2. La tasa de Dropout.



In [ ]:
def create_regularized_model(factor, rate):
    model = Sequential([
        Dense(64, kernel_regularizer=l2(factor), activation="relu", input_shape=(4,)),
        Dropout(rate),
        Dense(128, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(128, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(128, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(64, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(64, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(64, kernel_regularizer=l2(factor), activation="relu"),
        Dropout(rate),
        Dense(3, activation='softmax')
    ])
    return model

Vamos a crear un model con argumentos L2 con el factor 0.0001 y un Dropout rate de 0.3


In [ ]:
model = create_regularized_model (1e-5, 0.3)

model.summary()

El modelo regularizado puede ser entrenado tal como construimos el primer modelo.


In [ ]:
# Configurar el modelo usando la función model.compile()

model.compile( optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['categorical_accuracy'])

# Ahora, entrenamos el modelo con fit()

history = model.fit( x_train, y_train, epochs = 200, validation_split = 0.25, batch_size = 40, verbose = 2)


In [ ]:
# Volvemos a evaluar el modelo

plot_metric(history, 'loss')

Si bien el overfitting no esta completamente arreglado, hay una mejora significativa comparada con el modelo sin regularización.

Por lo que volvemos a evaluar el modelo.


In [ ]:
model.evaluate(x_test, y_test, verbose = 2)

### Optimización

In [ ]:
pip install tensorflow-addons


In [ ]:
pip install tensorflow tensorflow-addons adabelief-tf


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.optimizers import RectifiedAdam
import matplotlib.pyplot as plt

# Cargar y preprocesar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalizar los valores de píxeles a [0, 1]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Configuración de hiperparámetros
learning_rate = 0.001
batch_size = 64
epochs = 10

# Construir el modelo de la red neuronal
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Aplanar la imagen
    layers.Dense(128),  # Capa completamente conectada sin activación aquí
    layers.Activation('relu'),  # Añadir la activación como una capa separada
    layers.Dropout(0.5),  # Regularización por abandono
    layers.Dense(10, activation='softmax')  # Capa de salida con activación Softmax para clasificación multiclase
])

# Compilar el modelo con diferentes optimizadores y la función de pérdida de entropía cruzada categórica
optimizers = [
    tf.keras.optimizers.Adam(learning_rate=learning_rate),
    RectifiedAdam(learning_rate=learning_rate),
    tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)  # AMSGrad
]

# Entrenar el modelo con diferentes optimizadores y funciones de activación
for optimizer in optimizers:
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    # Evaluar el modelo en el conjunto de prueba
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"\nOptimizer: {optimizer.get_config()['name']}")
    print(f"Test Accuracy: {test_acc}")

    # Graficar la pérdida y la precisión durante el entrenamiento
    plt.figure(figsize=(12, 4))

    # Graficar la pérdida
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Graficar la precisión
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()
